In [ ]:
import json

import numpy as np
import pandas as pd

In [ ]:
with open("data/json/frames.json") as f:
    tmp = json.load(f)

In [ ]:
for turn in tmp[0]["turns"]:
    print("-" * 80)
    print(turn["text"])

In [ ]:
for i in range(10):
    print(tmp[i].get("turns", [])[0].get("text", ""))

In [ ]:
list(tmp[0].keys())

In [ ]:
tmp[0]["turns"][0]["text"]

In [ ]:
print(json.dumps(tmp[0], indent=2))

In [ ]:
res = []
for i in tmp:
    for j in i["turns"][0]["labels"]["acts_without_refs"]:
        res.append(len(j))

np.min(res), np.max(res)

In [ ]:
from collections import defaultdict

keys = defaultdict(set)
for i in tmp:
    for j in i["turns"][0]["labels"]["acts_without_refs"]:
        for k in j["args"]:
            try:
                keys[k["key"]].add(k.get("val", ""))
            except:
                print(k)

In [ ]:
list(keys.keys())

In [ ]:
print(json.dumps(tmp[0]["turns"][0]["labels"]["acts_without_refs"], indent=2))

In [ ]:
label_to_entity = {
    "or_city": "from_city",
    "dst_city": "to_city",
    "str_date": "from_dt",
    "end_date": "to_dt",
    "budget": "budget"
}

intent_name = "book_flight"

def frame_to_utterance(turn, intent_name, label_to_entity):
    """"""
    text = turn["text"]

    entity_labels = []
    for i in turn["labels"]["acts_without_refs"]:
        for l in i["args"]:
            k = l["key"]
            v = l["val"]

            if k and v:
                if k in label_to_entity.keys():
                    start_char_index = text.lower().find(v.lower())
                    if start_char_index == -1:
                        continue
                    send_char_index = start_char_index + len(v)

                    entity_labels.append({
                        "startCharIndex": start_char_index,
                        "endCharIndex": send_char_index,
                        "entityName": label_to_entity[k],
                    })
    
    res = {
        "text": text,
        "intentName": intent_name,
        "entityLabels": entity_labels,
    }
    return res

res = frame_to_utterance(tmp[129]["turns"][0], intent_name, label_to_entity)
print(json.dumps(res, indent=2))

In [ ]:
text = "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700."
text[27:35], text[41:48], text[52:77], text[117:121]

In [ ]:
res = []
for frame in tmp:
    # res.append(frame_to_utterance(frame["turns"][0], intent_name, label_to_entity))
    for turn in frame["turns"]:
        if turn["author"] == "user":
            res.append(frame_to_utterance(turn, intent_name, label_to_entity))

In [ ]:
df = pd.DataFrame(res)
df["entity_total_nb"] = df["entityLabels"].apply(len)
for entity_name in label_to_entity.values():
    df[f"{entity_name}_nb"] = df["entityLabels"].apply(lambda x: len(list(filter(lambda x1: x1["entityName"] == entity_name, x))))

df[["entity_total_nb"] + [f"{i}_nb" for i in label_to_entity.values()]].describe().T

In [ ]:
utterances = list(filter(lambda x: len(x["entityLabels"]) == len(label_to_entity), res))

print(len(utterances))

for utterance in utterances[:4]:
    print(json.dumps(utterance, indent=2))

In [ ]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

In [ ]:
from dotenv import load_dotenv, set_key

load_dotenv("../luis.env")

In [ ]:
LUIS_AUTH_KEY = os.getenv("LUIS_AUTH_KEY")
LUIS_AUTH_ENDPOINT = os.getenv("LUIS_AUTH_ENDPOINT")

LUIS_PRED_KEY = os.getenv("LUIS_PRED_KEY")
LUIS_PRED_ENDPOINT = os.getenv("LUIS_PRED_ENDPOINT")

In [ ]:
# On s'authentifie
client = LUISAuthoringClient(LUIS_AUTH_ENDPOINT, CognitiveServicesCredentials(LUIS_AUTH_KEY))

In [ ]:
# We use a UUID to avoid name collisions.
app_name = "p10_flight_booking"
app_version = "0.1"
intent_name = "book_flight"

In [ ]:
# define app basics
app_def = ApplicationCreateObject(name=app_name, initial_version_id=app_version, culture='en-us')

# create app
app_id = client.apps.add(app_def)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

In [ ]:
set_key("../luis.env", "LUIS_APP_ID", app_id)

In [ ]:
client.model.add_intent(app_id, app_version, intent_name)

In [ ]:
# Add Prebuilt entity
client.model.add_prebuilt(app_id, app_version, prebuilt_extractor_names=["number", "datetimeV2", "geographyV2"])

In [ ]:
entity_to_id = {
    "from_city": "",
    "to_city": "",
    "from_dt": "",
    "to_dt": "",
    "budget": ""
}

In [ ]:
for entity_name in entity_to_id.keys():
    # add entity to app
    entity_to_id[entity_name] = client.model.add_entity(
        app_id,
        app_version,
        name=entity_name
    )

entity_to_id

In [ ]:
# add phrase list to app
client.features.add_phrase_list(
    app_id,
    app_version,
    {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "from_phrase_list",
        "phrases": "from,start at,begin from,leave"
    }
)

# add phrase list to app
client.features.add_phrase_list(
    app_id,
    app_version,
    {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "to_phrase_list",
        "phrases": "to,arrive,land at,go,going,stay,heading"
    }
)

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"model_name": "geographyV2", "is_required": True}
)

# add phrase list as feature to subentity model
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"feature_name": "to_phrase_list", "model_name": None}
)

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"model_name": "geographyV2", "is_required": True}
)

# add phrase list as feature to subentity model
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"feature_name": "to_phrase_list", "model_name": None}
)

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["from_dt"],
    {"model_name": "datetimeV2", "is_required": True}
)

# client.features.add_entity_feature(
#     app_id,
#     app_version,
#     entity_to_id["from_dt"],
#     {"feature_name": "from_phrase_list", "model_name": None}
# )

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_dt"],
    {"model_name": "datetimeV2", "is_required": True}
)

# client.features.add_entity_feature(
#     app_id,
#     app_version,
#     entity_to_id["to_dt"],
#     {"feature_name": "to_phrase_list", "model_name": None}
# )

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["budget"],
    {"model_name": "number", "is_required": True}
)

In [ ]:
for utterance in utterances:
    try:
        client.examples.add(app_id, app_version, utterance, { "enableNestedChildren": True })
    except:
        print(json.dumps(utterance, indent=2))

In [ ]:
import time

client.train.train_version(app_id, app_version)
waiting = True
while waiting:
    info = client.train.get_status(app_id, app_version)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False

In [ ]:
responseEndpointInfo = client.apps.publish(app_id, app_version, is_staging=False)

In [ ]:
runtime_credentials = CognitiveServicesCredentials(LUIS_PRED_KEY)
client_runtime = LUISRuntimeClient(endpoint=LUIS_PRED_ENDPOINT, credentials=runtime_credentials)

In [ ]:
app_id = "3cbee687-5dc7-44e0-8574-cf85d03fe0ab"

In [ ]:
# Production == slot name
prediction_request = { "query" : "i want to go to london the 25/12/2021 and stay one week" }

prediction_response = client_runtime.prediction.get_slot_prediction(app_id, "Production", prediction_request)
print("Top intent: {}".format(prediction_response.prediction.top_intent))
print("Sentiment: {}".format (prediction_response.prediction.sentiment))
print("Intents: ")

for intent in prediction_response.prediction.intents:
    print("\t{}".format (json.dumps (intent)))
print("Entities: {}".format (prediction_response.prediction.entities))